In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data as data

# import torchvision.transforms as transforms
import torchvision
from torchvision import transforms, datasets

from sklearn import decomposition
from sklearn import manifold
from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np

import copy
import random
import time
from tqdm import tqdm

In [ ]:
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("Device:",device)

Device: cuda


In [ ]:
SEED = 100

torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True
random.seed(SEED)
np.random.seed(SEED)

In [ ]:
AUGMENTATION = False
NORMALIZATION = True

if AUGMENTATION :
  print("AUGMENTATION")
  train_transform = transforms.Compose([
                              transforms.RandomHorizontalFlip(),
                              transforms.RandomVerticalFlip(),
                              transforms.ColorJitter(brightness=2),
                              transforms.Resize((32,32)),
                              transforms.ToTensor()
                                        ])

  test_transform = transforms.Compose([
                            transforms.Resize((32,32)),
                             transforms.ToTensor()
                                       ])

elif AUGMENTATION and NORMALIZATION:
  print("AUGMENTATION and NORMALIZATION")
  train_transform = transforms.Compose([
                              transforms.RandomHorizontalFlip(),
                              transforms.RandomVerticalFlip(),
                              transforms.ColorJitter(brightness=2),
                              transforms.Resize((32,32)),
                              transforms.ToTensor(),
                              transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
                                        ])

  test_transform = transforms.Compose([
                             transforms.Resize((32,32)),
                             transforms.ToTensor(),
                             transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
                                       ])

elif NORMALIZATION and (not AUGMENTATION):
  print("No AUGMENTATION and NORMALIZATION")
  train_transform = transforms.Compose([
       transforms.Resize((32,32)),
       transforms.ToTensor(),
       transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
       ])

  test_transform = transforms.Compose([
       transforms.Resize((32,32)),
       transforms.ToTensor(),
       transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

elif (not NORMALIZATION) and (not AUGMENTATION):
  print("No AUGMENTATION and  No NORMALIZATION")
  train_transform = transforms.Compose(
      [transforms.Resize((32,32)),transforms.ToTensor()])

  test_transform = transforms.Compose(
      [transforms.Resize((32,32)),transforms.ToTensor()])

No AUGMENTATION and NORMALIZATION


In [ ]:
CIFAR = False

if CIFAR:
  trainset = torchvision.datasets.CIFAR10(root='/content/drive/My Drive/CV-P3/CIFAR-data', train=True,
                                          download=True, transform=train_transform)

  testset = torchvision.datasets.CIFAR10(root='/content/drive/My Drive/CV-P3/CIFAR-data', train=False,
                                        download=True, transform=test_transform)

  print("Type - CIFAR")
else:

  TRAIN_PATH= '/content/gdrive/MyDrive/DataCVproject/train'
  TEST_PATH = '/content/gdrive/MyDrive/DataCVproject/test'

  trainset = datasets.ImageFolder(root= TRAIN_PATH, transform=train_transform)
  testset = datasets.ImageFolder(root= TEST_PATH, transform=test_transform)

  print("Type - FDDB")


print("Length of Trainset - ",len(trainset))
print("Length of Testset - ",len(testset))

Type - FDDB
Length of Trainset -  2000
Length of Testset -  200


In [ ]:
VAL_SPLIT = 0.9 # 90%- Train, 10%- Validation

train_examples = int(len(trainset) * VAL_SPLIT)
valid_examples = len(trainset) - train_examples



train_data_, valid_data_ = data.random_split(trainset,
                                          [train_examples,valid_examples])



print(f'Complete data')
print(f'Images in training data: {len(train_data_)}')
print(f'Images in validation data: {len(valid_data_)}')
print(f'Images in test data: {len(testset)}')


valid_data_ = copy.deepcopy(valid_data_) # create a deep copy to avoid changing main object
valid_data_.dataset.transform = test_transform # apply test transforms to have consistency with test data


BATCH_SIZE = 32 # data batch_size

train_data = data.DataLoader(train_data_,
                                shuffle = True,
                                batch_size = BATCH_SIZE)

valid_data = data.DataLoader(valid_data_,
                                batch_size = BATCH_SIZE)

test_data = data.DataLoader(testset,
                                batch_size = BATCH_SIZE)

print(f'Batch Size: {BATCH_SIZE}')
print(f'Batches in training data: {len(train_data)}')
print(f'Batches in validation data: {len(valid_data)}')
print(f'Batches in test data: {len(test_data)}')

Complete data
Images in training data: 1800
Images in validation data: 200
Images in test data: 200
Batch Size: 32
Batches in training data: 57
Batches in validation data: 7
Batches in test data: 7


In [ ]:
class LeNet_V(nn.Module):
    def __init__(self, no_output):
        super().__init__()

        #first conv2d layer - [3,6,5]
        self.conv1 = nn.Conv2d(in_channels = 3,
                               out_channels = 6,
                               kernel_size = 5)

        nn.init.xavier_uniform_(self.conv1.weight)
        self.conv1_bn = nn.BatchNorm2d(6)

        #second conv2d layer - [6,16,5]
        self.conv2 = nn.Conv2d(in_channels = 6,
                               out_channels = 16,
                               kernel_size = 5)

        nn.init.xavier_uniform_(self.conv2.weight)
        self.conv2_bn = nn.BatchNorm2d(16)

        #fully connected layers
        self.fc_1 = nn.Linear(16 * 5 * 5, 120)
        nn.init.xavier_uniform_(self.fc_1.weight)
        self.fc_1_bn = nn.BatchNorm1d(120)

        self.fc_2 = nn.Linear(120, 84)
        nn.init.xavier_uniform_(self.fc_2.weight)
        self.fc_2_bn = nn.BatchNorm1d(84)

        self.fc_3 = nn.Linear(84, no_output)
        nn.init.xavier_uniform_(self.fc_3.weight)

    # Forward pass
    def forward(self, x):

        #x = [batch size, 3, 32, 32]

        x = F.relu(F.max_pool2d(self.conv1_bn(self.conv1(x)), kernel_size = 2))

        #x = [batch size, 6, 14, 14]

        x = F.relu(F.max_pool2d(self.conv2_bn(self.conv2(x)), kernel_size = 2))

        #x = [batch size, 16, 5, 5]


        x = x.view(x.shape[0], -1)

        #x = [batch size, 16*4*4 = 256]

        x = F.relu(self.fc_1_bn(self.fc_1(x)))

        #x = [batch size, 120]

        x = F.relu(self.fc_2_bn(self.fc_2(x)))

        x = self.fc_3(x)

        #x = [batch size, output dim]

        return x


In [ ]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

In [ ]:
if CIFAR:
  NO_OUTPUT = 10
else:
  NO_OUTPUT = 2

model = LeNet_V(NO_OUTPUT)

In [ ]:
print(f'The model has {count_parameters(model):,} trainable parameters')
print(model)

The model has 61,778 trainable parameters
LeNet_V(
  (conv1): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv1_bn): BatchNorm2d(6, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (conv2_bn): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc_1): Linear(in_features=400, out_features=120, bias=True)
  (fc_1_bn): BatchNorm1d(120, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc_2): Linear(in_features=120, out_features=84, bias=True)
  (fc_2_bn): BatchNorm1d(84, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc_3): Linear(in_features=84, out_features=2, bias=True)
)


In [ ]:
def calculate_accuracy(y_pred, y):
    top_pred = y_pred.argmax(1, keepdim = True)
    correct = top_pred.eq(y.view_as(top_pred)).sum()
    acc = correct.float() / y.shape[0]
    return acc

In [ ]:
def train(model, data_iterator, optimizer, criterion, device):

    epoch_loss = 0
    epoch_acc = 0

    # model in train mode
    model.train()

    for (x, y) in data_iterator:

        # load variable in device memory
        x = x.to(device)
        y = y.to(device)

        # make gradient as zero
        optimizer.zero_grad()

        # predict output
        y_pred = model(x)

        # calculate loss
        loss = criterion(y_pred, y)

        # backward propagation
        loss.backward()

        # calculate accuracy
        acc = calculate_accuracy(y_pred, y)

        # take step as per learning rate & regularization
        optimizer.step()

        # calculate losses
        epoch_loss += loss.item()
        epoch_acc += acc.item()

    return epoch_loss / len(data_iterator), epoch_acc / len(data_iterator)


In [ ]:
def evaluate(model, data_iterator, criterion, device):

    epoch_loss = 0
    epoch_acc = 0

    # model in evaluation mode
    model.eval()

    with torch.no_grad():

        for (x, y) in data_iterator:

            x = x.to(device)
            y = y.to(device)

            y_pred = model(x)

            loss = criterion(y_pred, y)

            acc = calculate_accuracy(y_pred, y)

            epoch_loss += loss.item()
            epoch_acc += acc.item()

    return epoch_loss / len(data_iterator), epoch_acc / len(data_iterator)

In [ ]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [ ]:
def get_loss_before_train(model, data_iterator, criterion, device):

    epoch_loss = 0

    for (x, y) in tqdm(data_iterator):

        x = x.to(device)
        y = y.to(device)

        y_pred = model(x)

        loss = criterion(y_pred, y)

        epoch_loss += loss.item()

    return epoch_loss/len(data_iterator)

In [ ]:
def model_loss(train_data_iterator, device,reg=0):
  NO_OUTPUT = 2
  model = LeNet_V(NO_OUTPUT)

  criterion = nn.CrossEntropyLoss()

  model = model.to(device)
  criterion = criterion.to(device)

  loss_ = get_loss_before_train(model, train_data_iterator, criterion, device)

  return loss_

In [ ]:
def plot_metrics(TRAIN_ACC,VAL_ACC,TRAIN_LOSS,VAL_LOSS):
  plt.plot(np.asarray(TRAIN_ACC),'r',label="Training")
  plt.plot(np.asarray(VAL_ACC),'b',label="Validation")
  plt.title("Training & Validation Accuracy")
  plt.xlabel("Epochs")
  plt.ylabel("Accuracy")
  plt.legend(loc='best')
  plt.show()

  plt.plot(np.asarray(TRAIN_LOSS),'r',label="Training")
  plt.plot(np.asarray(VAL_LOSS),'b',label="Validation")
  plt.title("Training & Validation Loss")
  plt.xlabel("Epochs")
  plt.ylabel("Loss")
  plt.legend(loc='best')
  plt.show()

In [ ]:
loss_ = model_loss(train_data,device,reg=0)
print("Loss: ",loss_)

100%|██████████| 57/57 [00:20<00:00,  2.73it/s]

Loss:  1.2971892670581215


In [ ]:
loss_ = model_loss(train_data,device,reg=10**3)
print("Loss: ",loss_)

100%|██████████| 57/57 [00:02<00:00, 21.10it/s]

Loss:  1.0157523029728939


In [ ]:
def TRAIN(train_data_iterator,valid_data_iterator,device,optimizer,no_epochs,LR=0,reg=0):
  NO_OUTPUT = 2
  model = LeNet_V(NO_OUTPUT)

  if optimizer == 'SGD':
    optimizer = optim.SGD(model.parameters(), lr=LR, weight_decay=reg, momentum = 0.9)
  elif optimizer == 'Adam':
    optimizer = optim.Adam(model.parameters(), lr=LR, weight_decay=reg)

  criterion = nn.CrossEntropyLoss()

  model = model.to(device)
  criterion = criterion.to(device)

  EPOCHS = no_epochs

  best_valid_loss = float('inf')

  TRAIN_ACC = []
  VAL_ACC = []
  TRAIN_LOSS = []
  VAL_LOSS = []
  for epoch in tqdm(range(EPOCHS)):

      start_time = time.time()

      train_loss, train_acc = train(model, train_data_iterator, optimizer, criterion, device)
      valid_loss, valid_acc = evaluate(model, valid_data_iterator, criterion, device)

      if valid_loss < best_valid_loss:
          best_valid_loss = valid_loss
          torch.save(model.state_dict(), 'cv-babysitting-model.pt')

      end_time = time.time()

      epoch_mins, epoch_secs = epoch_time(start_time, end_time)

      TRAIN_ACC.append(train_acc)
      TRAIN_LOSS.append(train_loss)
      VAL_ACC.append(valid_acc)
      VAL_LOSS.append(valid_loss)
      print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
      print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
      print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')


In [ ]:
def FINE_TRAIN(train_data_iterator,valid_data_iterator,device,optimizer,no_epochs,LR=0,reg=0):
  NO_OUTPUT = 2
  model = LeNet_V(NO_OUTPUT)

  if optimizer == 'SGD':
    optimizer = optim.SGD(model.parameters(), lr=LR, weight_decay=reg, momentum = 0.9)
  elif optimizer == 'Adam':
    optimizer = optim.Adam(model.parameters(), lr=LR, weight_decay=reg)

  criterion = nn.CrossEntropyLoss()

  model = model.to(device)
  criterion = criterion.to(device)

  EPOCHS = no_epochs

  best_valid_loss = float('inf')

  TRAIN_ACC = []
  VAL_ACC = []
  TRAIN_LOSS = []
  VAL_LOSS = []
  for epoch in tqdm(range(EPOCHS)):

      start_time = time.time()

      train_loss, train_acc = train(model, train_data_iterator, optimizer, criterion, device)
      valid_loss, valid_acc = evaluate(model, valid_data_iterator, criterion, device)

      if valid_loss < best_valid_loss:
          best_valid_loss = valid_loss
          torch.save(model.state_dict(), 'cv-babysitting-model.pt')

      end_time = time.time()

      epoch_mins, epoch_secs = epoch_time(start_time, end_time)

      TRAIN_ACC.append(train_acc)
      TRAIN_LOSS.append(train_loss)
      VAL_ACC.append(valid_acc)
      VAL_LOSS.append(valid_loss)
      print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
      print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
      print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')
      print(f'LR: {LR} | Reg: {reg}')


In [ ]:
TRAIN(train_data,valid_data,device,optimizer='SGD',no_epochs=200,LR=10**-3,reg=0)


  0%|          | 1/200 [00:03<10:49,  3.26s/it]

Epoch: 01 | Epoch Time: 0m 3s
	Train Loss: 0.263 | Train Acc: 89.64%
	 Val. Loss: 0.073 |  Val. Acc: 100.00%


  1%|          | 2/200 [00:06<10:13,  3.10s/it]

Epoch: 02 | Epoch Time: 0m 2s
	Train Loss: 0.040 | Train Acc: 100.00%
	 Val. Loss: 0.035 |  Val. Acc: 100.00%


  2%|▏         | 3/200 [00:09<10:04,  3.07s/it]

...

Epoch: 198 | Epoch Time: 0m 2s
	Train Loss: 0.000 | Train Acc: 100.00%
	 Val. Loss: 0.000 |  Val. Acc: 100.00%


100%|█████████▉| 199/200 [09:51<00:03,  3.01s/it]

Epoch: 199 | Epoch Time: 0m 3s
	Train Loss: 0.000 | Train Acc: 100.00%
	 Val. Loss: 0.000 |  Val. Acc: 100.00%


100%|██████████| 200/200 [09:54<00:00,  2.97s/it]

Epoch: 200 | Epoch Time: 0m 3s
	Train Loss: 0.000 | Train Acc: 100.00%
	 Val. Loss: 0.000 |  Val. Acc: 100.00%


In [ ]:
TRAIN(train_data,valid_data,device,optimizer='SGD',no_epochs=10,LR=10**-6,reg=10**-6)


 10%|█         | 1/10 [00:03<00:28,  3.15s/it]

Epoch: 01 | Epoch Time: 0m 3s
	Train Loss: 0.440 | Train Acc: 79.99%
	 Val. Loss: 0.433 |  Val. Acc: 77.23%


 20%|██        | 2/10 [00:06<00:24,  3.08s/it]

Epoch: 02 | Epoch Time: 0m 3s
	Train Loss: 0.438 | Train Acc: 80.54%
	 Val. Loss: 0.471 |  Val. Acc: 73.66%


 30%|███       | 3/10 [00:09<00:21,  3.06s/it]

...

Epoch: 08 | Epoch Time: 0m 2s
	Train Loss: 0.424 | Train Acc: 81.30%
	 Val. Loss: 0.441 |  Val. Acc: 76.79%


 90%|█████████ | 9/10 [00:26<00:02,  2.95s/it]

Epoch: 09 | Epoch Time: 0m 2s
	Train Loss: 0.416 | Train Acc: 81.52%
	 Val. Loss: 0.484 |  Val. Acc: 70.09%


100%|██████████| 10/10 [00:29<00:00,  2.99s/it]

Epoch: 10 | Epoch Time: 0m 2s
	Train Loss: 0.419 | Train Acc: 81.41%
	 Val. Loss: 0.427 |  Val. Acc: 78.57%


In [ ]:
TRAIN(train_data,valid_data,device,optimizer='SGD',no_epochs=10,LR=10**6,reg=10**-6)


 10%|█         | 1/10 [00:02<00:26,  2.98s/it]

Epoch: 01 | Epoch Time: 0m 2s
	Train Loss: nan | Train Acc: 95.23%
	 Val. Loss: nan |  Val. Acc: 100.00%


 20%|██        | 2/10 [00:05<00:23,  2.92s/it]

Epoch: 02 | Epoch Time: 0m 2s
	Train Loss: nan | Train Acc: 100.00%
	 Val. Loss: nan |  Val. Acc: 100.00%


 30%|███       | 3/10 [00:08<00:20,  2.92s/it]

...

Epoch: 08 | Epoch Time: 0m 2s
	Train Loss: nan | Train Acc: 100.00%
	 Val. Loss: nan |  Val. Acc: 100.00%


 90%|█████████ | 9/10 [00:26<00:02,  2.93s/it]

Epoch: 09 | Epoch Time: 0m 2s
	Train Loss: nan | Train Acc: 100.00%
	 Val. Loss: nan |  Val. Acc: 100.00%


100%|██████████| 10/10 [00:29<00:00,  2.93s/it]

Epoch: 10 | Epoch Time: 0m 2s
	Train Loss: nan | Train Acc: 100.00%
	 Val. Loss: nan |  Val. Acc: 100.00%


In [ ]:
TRAIN(train_data,valid_data,device,optimizer='SGD',no_epochs=10,LR=3*10**-3,reg=10**-6)


 10%|█         | 1/10 [00:03<00:27,  3.00s/it]

Epoch: 01 | Epoch Time: 0m 3s
	Train Loss: 0.112 | Train Acc: 96.93%
	 Val. Loss: 0.016 |  Val. Acc: 100.00%


 20%|██        | 2/10 [00:05<00:23,  2.96s/it]

Epoch: 02 | Epoch Time: 0m 2s
	Train Loss: 0.010 | Train Acc: 100.00%
	 Val. Loss: 0.010 |  Val. Acc: 100.00%


 30%|███       | 3/10 [00:08<00:20,  2.96s/it]

...

Epoch: 08 | Epoch Time: 0m 2s
	Train Loss: 0.002 | Train Acc: 100.00%
	 Val. Loss: 0.003 |  Val. Acc: 100.00%


 90%|█████████ | 9/10 [00:26<00:02,  2.92s/it]

Epoch: 09 | Epoch Time: 0m 2s
	Train Loss: 0.002 | Train Acc: 100.00%
	 Val. Loss: 0.003 |  Val. Acc: 100.00%


100%|██████████| 10/10 [00:29<00:00,  2.94s/it]

Epoch: 10 | Epoch Time: 0m 2s
	Train Loss: 0.002 | Train Acc: 100.00%
	 Val. Loss: 0.003 |  Val. Acc: 100.00%


In [ ]:
TRAIN(train_data,valid_data,device,optimizer='SGD',no_epochs=10,LR=10**-3,reg=10**-3)


 10%|█         | 1/10 [00:02<00:26,  2.93s/it]

Epoch: 01 | Epoch Time: 0m 2s
	Train Loss: 0.286 | Train Acc: 87.88%
	 Val. Loss: 0.071 |  Val. Acc: 100.00%


 20%|██        | 2/10 [00:05<00:23,  2.97s/it]

Epoch: 02 | Epoch Time: 0m 2s
	Train Loss: 0.038 | Train Acc: 100.00%
	 Val. Loss: 0.035 |  Val. Acc: 100.00%


 30%|███       | 3/10 [00:08<00:20,  2.95s/it]

...

Epoch: 08 | Epoch Time: 0m 2s
	Train Loss: 0.008 | Train Acc: 100.00%
	 Val. Loss: 0.010 |  Val. Acc: 100.00%


 90%|█████████ | 9/10 [00:26<00:02,  2.98s/it]

Epoch: 09 | Epoch Time: 0m 2s
	Train Loss: 0.007 | Train Acc: 100.00%
	 Val. Loss: 0.008 |  Val. Acc: 100.00%


100%|██████████| 10/10 [00:29<00:00,  2.98s/it]

Epoch: 10 | Epoch Time: 0m 2s
	Train Loss: 0.006 | Train Acc: 100.00%
	 Val. Loss: 0.008 |  Val. Acc: 100.00%


In [ ]:
TRAIN(train_data,valid_data,device,optimizer='SGD',no_epochs=10,LR=10**-3,reg=10**3)


 10%|█         | 1/10 [00:03<00:27,  3.00s/it]

Epoch: 01 | Epoch Time: 0m 3s
	Train Loss: 0.702 | Train Acc: 55.43%
	 Val. Loss: 0.694 |  Val. Acc: 0.00%


 20%|██        | 2/10 [00:05<00:23,  2.97s/it]

Epoch: 02 | Epoch Time: 0m 2s
	Train Loss: 0.693 | Train Acc: 78.95%
	 Val. Loss: 0.693 |  Val. Acc: 100.00%


 30%|███       | 3/10 [00:08<00:20,  2.96s/it]

...

Epoch: 08 | Epoch Time: 0m 2s
	Train Loss: 0.693 | Train Acc: 100.00%
	 Val. Loss: 0.693 |  Val. Acc: 100.00%


 90%|█████████ | 9/10 [00:26<00:02,  2.94s/it]

Epoch: 09 | Epoch Time: 0m 2s
	Train Loss: 0.693 | Train Acc: 100.00%
	 Val. Loss: 0.693 |  Val. Acc: 100.00%


100%|██████████| 10/10 [00:29<00:00,  2.95s/it]

Epoch: 10 | Epoch Time: 0m 2s
	Train Loss: 0.693 | Train Acc: 100.00%
	 Val. Loss: 0.693 |  Val. Acc: 100.00%


In [ ]:
TRAIN(train_data,valid_data,device,optimizer='SGD',no_epochs=10,LR=10**-3,reg=10**4)


 10%|█         | 1/10 [00:02<00:26,  2.99s/it]

Epoch: 01 | Epoch Time: 0m 2s
	Train Loss: nan | Train Acc: 89.53%
	 Val. Loss: nan |  Val. Acc: 100.00%


 20%|██        | 2/10 [00:05<00:23,  2.97s/it]

Epoch: 02 | Epoch Time: 0m 2s
	Train Loss: nan | Train Acc: 100.00%
	 Val. Loss: nan |  Val. Acc: 100.00%


 30%|███       | 3/10 [00:08<00:20,  2.94s/it]

...

Epoch: 08 | Epoch Time: 0m 2s
	Train Loss: nan | Train Acc: 100.00%
	 Val. Loss: nan |  Val. Acc: 100.00%


 90%|█████████ | 9/10 [00:26<00:02,  2.92s/it]

Epoch: 09 | Epoch Time: 0m 2s
	Train Loss: nan | Train Acc: 100.00%
	 Val. Loss: nan |  Val. Acc: 100.00%


100%|██████████| 10/10 [00:29<00:00,  2.93s/it]

Epoch: 10 | Epoch Time: 0m 2s
	Train Loss: nan | Train Acc: 100.00%
	 Val. Loss: nan |  Val. Acc: 100.00%


In [ ]:
max_count = 20

for count in tqdm(range(max_count)):
  LR = 10**random.uniform(-5,5)
  reg = 10**random.uniform(-3,-6)
  FINE_TRAIN(train_data,valid_data,device,optimizer='SGD',no_epochs=5,LR=LR,reg=reg)
  print("-----------------------------------------------------------")

 20%|██        | 1/5 [00:02<00:11,  2.94s/it]

Epoch: 01 | Epoch Time: 0m 2s
	Train Loss: 0.521 | Train Acc: 75.77%
	 Val. Loss: 0.274 |  Val. Acc: 95.09%
LR: 0.0002862151057412364 | Reg: 4.317367186961589e-05



 40%|████      | 2/5 [00:05<00:08,  2.96s/it]

Epoch: 02 | Epoch Time: 0m 2s
	Train Loss: 0.173 | Train Acc: 98.79%
	 Val. Loss: 0.143 |  Val. Acc: 100.00%
LR: 0.0002862151057412364 | Reg: 4.317367186961589e-05



 60%|██████    | 3/5 [00:08<00:05,  2.95s/it]

...

Epoch: 03 | Epoch Time: 0m 2s
	Train Loss: 0.000 | Train Acc: 100.00%
	 Val. Loss: 0.000 |  Val. Acc: 100.00%
LR: 0.3033366120337391 | Reg: 1.2206764923453576e-05



 80%|████████  | 4/5 [00:11<00:02,  2.92s/it]

Epoch: 04 | Epoch Time: 0m 2s
	Train Loss: 0.000 | Train Acc: 100.00%
	 Val. Loss: 0.000 |  Val. Acc: 100.00%
LR: 0.3033366120337391 | Reg: 1.2206764923453576e-05



100%|██████████| 20/20 [04:55<00:00, 14.79s/it]

Epoch: 05 | Epoch Time: 0m 2s
	Train Loss: 0.000 | Train Acc: 100.00%
	 Val. Loss: 0.000 |  Val. Acc: 100.00%
LR: 0.3033366120337391 | Reg: 1.2206764923453576e-05
-----------------------------------------------------------


In [ ]:
max_count = 10

for count in tqdm(range(max_count)):
  LR = 10**random.uniform(-4,-2)
  reg = 10**random.uniform(-3,-5)
  FINE_TRAIN(train_data,valid_data,device,optimizer='SGD',no_epochs=5,LR=LR,reg=reg)
  print("-----------------------------------------------------------")

 20%|██        | 1/5 [00:02<00:11,  2.92s/it]

Epoch: 01 | Epoch Time: 0m 2s
	Train Loss: 0.085 | Train Acc: 96.44%
	 Val. Loss: 0.004 |  Val. Acc: 100.00%
LR: 0.00637802247761107 | Reg: 0.0005581739741424622



 40%|████      | 2/5 [00:05<00:08,  2.95s/it]

Epoch: 02 | Epoch Time: 0m 2s
	Train Loss: 0.003 | Train Acc: 100.00%
	 Val. Loss: 0.003 |  Val. Acc: 100.00%
LR: 0.00637802247761107 | Reg: 0.0005581739741424622



 60%|██████    | 3/5 [00:08<00:05,  2.94s/it]

...

Epoch: 03 | Epoch Time: 0m 3s
	Train Loss: 0.010 | Train Acc: 100.00%
	 Val. Loss: 0.011 |  Val. Acc: 100.00%
LR: 0.0020091551055966416 | Reg: 0.00031256055391324787



 80%|████████  | 4/5 [00:12<00:03,  3.04s/it]

Epoch: 04 | Epoch Time: 0m 3s
	Train Loss: 0.007 | Train Acc: 100.00%
	 Val. Loss: 0.008 |  Val. Acc: 100.00%
LR: 0.0020091551055966416 | Reg: 0.00031256055391324787



100%|██████████| 10/10 [02:28<00:00, 14.83s/it]

Epoch: 05 | Epoch Time: 0m 3s
	Train Loss: 0.006 | Train Acc: 100.00%
	 Val. Loss: 0.006 |  Val. Acc: 100.00%
LR: 0.0020091551055966416 | Reg: 0.00031256055391324787
-----------------------------------------------------------


In [ ]:
#LR = 0.007461627791541241
#reg = 0.0001580451045434134
#LR = 2.1388089166220053e-05
#reg = 4.508058651380181e-05
#LR=6.331944386333944e-05
#reg=4.287087587289784e-05
LR= 3.0121400777449763e-05
reg= 1.5889593971470228e-06

TRAIN(train_data,valid_data,device,optimizer='SGD',no_epochs=30,LR=LR,reg=reg)

  3%|▎         | 1/30 [00:03<01:27,  3.03s/it]

Epoch: 01 | Epoch Time: 0m 3s
	Train Loss: 0.639 | Train Acc: 63.54%
	 Val. Loss: 0.584 |  Val. Acc: 69.64%


  7%|▋         | 2/30 [00:06<01:24,  3.03s/it]

Epoch: 02 | Epoch Time: 0m 3s
	Train Loss: 0.548 | Train Acc: 72.86%
	 Val. Loss: 0.578 |  Val. Acc: 71.43%


 10%|█         | 3/30 [00:08<01:20,  2.98s/it]

...

Epoch: 28 | Epoch Time: 0m 2s
	Train Loss: 0.084 | Train Acc: 99.95%
	 Val. Loss: 0.106 |  Val. Acc: 100.00%


 97%|█████████▋| 29/30 [01:25<00:02,  2.94s/it]

Epoch: 29 | Epoch Time: 0m 2s
	Train Loss: 0.080 | Train Acc: 99.95%
	 Val. Loss: 0.101 |  Val. Acc: 100.00%


100%|██████████| 30/30 [01:28<00:00,  2.94s/it]

Epoch: 30 | Epoch Time: 0m 2s
	Train Loss: 0.078 | Train Acc: 99.95%
	 Val. Loss: 0.092 |  Val. Acc: 100.00%


In [ ]:
criterion = nn.CrossEntropyLoss()
criterion = criterion.to(device)

model.load_state_dict(torch.load('/content/cv-babysitting-model.pt'))
model = model.to(device)

In [ ]:
test_loss, test_acc = evaluate(model, test_data, criterion, device)
print(f'Test Accuracy: {test_acc} | Test Loss: {test_loss}')

Test Accuracy: 0.9910714285714286 | Test Loss: 0.15567311538117273
